# Setup and load data:

In [ ]:
%load_ext autoreload
%autoreload 2

from functions.functions import *
from functions.plotting import *
from functions.autoencoder import *
from functions.mdn import *
from keras import backend as K
from keras.callbacks import TerminateOnNaN, ModelCheckpoint
from keras.models import model_from_json
from livelossplot.keras import PlotLossesCallback
import livelossplot
import tensorflow as tf
import numpy as np
import os
import glob
from itertools import combinations
import matplotlib
plt.rcParams['animation.ffmpeg_path'] = '/gpfs/loomis/project/hep/demers/mnp3/conda_envs/choreo/bin/ffmpeg' # for using html5 video in Jupyter notebook
print(matplotlib.animation.writers.list()) # check that ffmpeg is loaded. if it's not there, use .to_jshtml() instead of .to_html5_video().

In [ ]:
setup_gpus()

In [ ]:
data = load_data('rnn_data/mariel_*')

# Check out the real data:

In [ ]:
frame = np.random.randint(0,data.full.X.shape[1]-50)
print("Starting from frame {}...".format(frame))

HTML(animate(data.full.X[:,frame:,:], frames=100))
# HTML(animate(data.selected.X[:,frame:,:], frames=100, edges=data.selected.edges, colors='black'))

# Build the autoencoder for poses

### Define the training dataset:

In [ ]:
# X_train = data.selected.X    # 15 joints
X_train = data.full.X[:,:int(data.full.X.shape[1]*.8),:]    # 53 joints (80% train)
X_test = data.full.X[:,int(data.full.X.shape[1]*.8):,:]    # 53 joints (20% test)
print("Training set size: {}".format(X_train.shape))
print("Test set size: {}".format(X_test.shape))

### Train without adding random (x,y) offsets:

In [ ]:
ae_nooffset = Autoencoder(n_verts=X_train.shape[0], latent_dim=32, n_layers=2, n_units=64, relu=True, dropout=False, add_random_offsets=False)
X_T = X_train.transpose((1,0,2))
ae_nooffset.model.fit(X_T, X_T, batch_size = 128, epochs = 50)

### Real data in blue, autoencoded predictions in orange (we want these to be very similar):

In [ ]:
n_frames = 100
frame = np.random.randint(0,X_test.shape[1]-n_frames)
print("Seeding with frame {}".format(frame))
nooffset_predictions = ae_nooffset.get_predictions(X_test, start_frame=frame, n_frames=n_frames)
HTML(animate_ghost(X_test[:,frame:,:], nooffset_predictions, frames=n_frames, ghost_shift=0.2))

### Train with adding random (x,y) offsets to the input data:

In [ ]:
ae_withoffset = Autoencoder(n_verts=X_train.shape[0], latent_dim=32, n_layers=2, n_units=64, relu=True, dropout=False, add_random_offsets=False)
X_T = X_train.transpose((1,0,2))
ae_withoffset.model.fit(X_T, X_T, batch_size = 128, epochs = 50)

### Real data in blue, autoencoded predictions in orange (we want these to be very similar):

In [ ]:
n_frames = 100
frame = np.random.randint(0,X_test.shape[1]-n_frames)
print("Seeding with frame {}".format(frame)) 
withoffset_predictions = ae_withoffset.get_predictions(X_test, start_frame=frame, n_frames=n_frames)
HTML(animate_ghost(X_test[:,frame:,:], withoffset_predictions, frames=n_frames, ghost_shift=0.2))

# Compare predictions with and without (x,y) offsets:

In [ ]:
n_frames = 100
frame = np.random.randint(0,X_test.shape[1]-n_frames)
print("Seeding with frame {}".format(frame)) 
nooffset_predictions = ae_nooffset.get_predictions(X_test, start_frame=frame, n_frames=n_frames)
withoffset_predictions = ae_withoffset.get_predictions(X_test, start_frame=frame, n_frames=n_frames)
HTML(animate_ghost(nooffset_predictions, withoffset_predictions, frames=n_frames, ghost_shift=0.2))